# *Красный трек*
## Задачи по теме Plotly

Решая задачи, ты научишься:

- строить гистограммы и статистические графики с помощью Plotly;
- использовать Plotly Express для создания интерактивных визуализаций;
- настраивать внешний вид графиков: цвета, размеры, шрифты;
- интерпретировать визуализации, чтобы получить полезную информацию из данных;
- создавать анимации и интерактивные элементы для улучшения пользовательского опыта.

Время решения: ~60 минут.

## Оценивание домашнего задания

**Система оценивания —** бинарная:

  - если все задачи решены корректно, без ошибок и полностью соответствуют стандартам кода на курсе, то задание выполнено и оценка — **10 баллов**;
  - если решения содержат ошибки или не соответствуют требованиям, то задание не выполнено и оценка — **0 баллов**.


**Проверка задания**

- Перед тем, как сдать задание, убедись, что твой код работает без ошибок и соответствует стандартам. Для этого используй автоматическую проверку.
- Загрузи задание в LMS. Ассистент проверит, соответствуют ли твои решения требованиям и целям домашнего задания, и выставит оценку.


**Доработка**

  - Если твое задание получило 0 баллов, его вернут на доработку через LMS с комментариями о том, что нужно исправить.

## Подготовка данных из онлайн-источников

Выполни ячейку ниже, чтобы загрузить данные для твоих задач.

Краткое описание загружаемых данных:

1.	**Propagated Solar Wind (1-Hour):** прогноз солнечного ветра с часовыми интервалами, включающий скорость и плотность частиц.
2.	**Boulder K-Index (1-Minute):** ежеминутные данные K-индекса, измеряющие геомагнитную активность.
3.	**Earthquake Data (USGS):** данные о землетрясениях (место, сила, время) за январь 2024 года. Они используются для мониторинга сейсмической активности, оценки риска и реагирования на чрезвычайные ситуации.
4.	**Ovation Aurora:** прогнозы северного и южного сияний, основанные на космической погоде.
5.	**Solar Wind Plasma (7-Day):** параметры солнечного ветра (скорость, плотность) за последние 7 дней.
6.	**Solar Cycle 25 F10.7:** прогнозы солнечной активности на основе радиоизлучения для 25-го солнечного цикла. Эти данные могут использоваться для изучения атмосферных явлений и прогнозирования появления северного и южного сияний.

Все эти данные используются для научных исследований и мониторинга природных явлений. Они играют важную роль в области космической погоды, сейсмологии и климатологии. Такие данные могут предупредить о потенциальных угрозах, связанных с солнечной активностью и геофизическими событиями.


In [ ]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import requests
from plotly.subplots import make_subplots

In [ ]:
# solar wind
url = 'https://services.swpc.noaa.gov/products/geospace/propagated-solar-wind-1-hour.json'
response = requests.get(url)
data = response.json()
columns = data[0]
solar_wind = pd.DataFrame(data[1:], columns=columns)
solar_wind['speed'] = pd.to_numeric(solar_wind['speed'])
solar_wind['density'] = pd.to_numeric(solar_wind['density'])
solar_wind['temperature'] = pd.to_numeric(solar_wind['temperature'])
solar_wind['time_tag'] = pd.to_datetime(solar_wind['time_tag'])
# k-index
url = 'https://services.swpc.noaa.gov/json/boulder_k_index_1m.json'
response = requests.get(url)
data = response.json()
k_index_df = pd.DataFrame(data)
k_index_df['time_tag'] = pd.to_datetime(k_index_df['time_tag'])
k_index_df.set_index('time_tag', inplace=True)
k_index_df = k_index_df.resample('40T').median().reset_index()
# earthquake
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-01-01&endtime=2024-02-01'
response = requests.get(url)
data = response.json()
earthquake_data = []
for feature in data['features']:
    properties = feature['properties']
    coordinates = feature['geometry']['coordinates']
    time = pd.to_datetime(properties['time'], unit='ms')
    if properties['mag'] is not None and properties['mag'] >= 0:
          earthquake_data.append({
              'longitude': coordinates[0],
              'latitude': coordinates[1],
              'time': time,
              'mag': properties['mag']
          })
earthquake = pd.DataFrame(earthquake_data)
min_time = earthquake['time'].min()
max_time = earthquake['time'].max()
new_times = pd.date_range(start=min_time, end=max_time, periods=100)
earthquake['time'] = earthquake['time'].apply(lambda x: min(new_times, key=lambda t: abs(t - x)))
# f10
url = 'https://services.swpc.noaa.gov/products/solar-cycle-25-f10-7-predicted-range.json'
f10 = requests.get(url).json()
f10 = pd.DataFrame(f10)
f10['time-tag'] = pd.to_datetime(f10['time-tag'])
# plasma
url = 'https://services.swpc.noaa.gov/products/solar-wind/plasma-7-day.json'
data = requests.get(url).json()
plasma = pd.DataFrame(data[1:], columns=data[0])
plasma['density'] = pd.to_numeric(plasma['density'], errors='coerce')
plasma['speed'] = pd.to_numeric(plasma['speed'], errors='coerce')
plasma['temperature'] = pd.to_numeric(plasma['temperature'], errors='coerce')
plasma = plasma.dropna(subset=['density', 'speed', 'temperature'])
# aurora
url = 'https://services.swpc.noaa.gov/json/ovation_aurora_latest.json'
response = requests.get(url)
aurora = response.json()
aurora = pd.DataFrame(aurora['coordinates'], columns=['Longitude', 'Latitude', 'Aurora'])

<ipython-input-2-7a32dd039cac>:18: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  k_index_df = k_index_df.resample('40T').median().reset_index()


### Задача 1

Вспышки на Солнце способны вызывать мощные выбросы плазмы солнечного ветра, которые приводят к геомагнитным бурям. Бури могут нарушить работу спутников, навигационных систем и даже повлиять на работу энергосетей на Земле. Проанализируй показатели солнечного ветра за последние 7 дней.

**Входные данные**
- `plasma`: датафрейм с данными
- `density`: плотность солнечного ветра (в частицах на кубический сантиметр).
- `speed`: скорость солнечного ветра (в километрах в секунду).
-	`temperature`: температура солнечного ветра (в кельвинах).

**Что нужно сделать**
-	Построй пузырьковую диаграмму с осями:
  - Ось $X$: скорость солнечного ветра (`speed`).
  -	Ось $Y$: температура солнечного ветра (`temperature`).
- Заголовок: «Bubble Chart of Solar Wind Plasma Data».
  - Ось $X$: «Speed ($\frac{km}{s}$)».
  - Ось $Y$: «Temperature ($K$)».
-	Размер пузырьков должен быть пропорционален плотности солнечного ветра (`density`).
-	Цвет пузырьков должен отражать плотность солнечного ветра. Заголовок для цветовой шкалы: «Density ($cm^{-3}$)».



In [ ]:
# Напиши свой код здесь


### Задача 2

Визуализируй данные K-индекса — ключевого показателя геомагнитной активности. Для этого используй временные метки и значения индекса.

**Примечание.**  K-индекс — это логарифмическая шкала, которую используют для измерения геомагнитных бурь. Он показывает интенсивность изменений магнитного поля Земли в трёхчасовом интервале. Значения K-индекса колеблются от 0 до 9:
- **0–2**: спокойная геомагнитная обстановка.
- **3–4**: слабые возмущения, возможные минимальные геомагнитные бури.
- **5–9**: магнитные бури разной степени интенсивности (от умеренных до экстремальных), которые могут спровоцировать сбой геосистем или  северное сияние.

**Входные данные**
- `k_index_df`: датафрейм с данными.
- `time_tag`: временная метка, указывающая точное время измерения (например, 2024-09-23T10:32:00).
-	`k_index`: числовое значение K-индекса, показывающее уровень геомагнитной активности (например, `3.3`).
- Данные представлены с интервалом в одну минуту.

**Что нужно сделать**  
- Построй гистограмму, отображающую значения K-индекса по временным меткам.
- Отрази значение K-индекса на цветовой палитре графика: низкие индексы отметь зелёным цветом, высокие — красным. Используй непрерывную палитру зелёно-жёлто-красного цвета.
-	Укажи заголовок графика и подпиши оси:
  -	Заголовок: «K Index Histogram (40-Minute Intervals)».
  -	Ось $X$: «Time» (дата и время).
  -	Ось $Y$: «K Index» (значение K-индекса).
-	Задай размер графика — 800x400 пикселей.

In [ ]:
# Напиши свой код здесь


### Задача 3

Исследуй характеристики солнечного ветра. Его скорость, плотность, температура и магнитные поля (компоненты $B_x$, $B_y$, $B_z$, и $B_t$) играют ключевую роль в изучении космической погоды и её воздействия на Землю, в том числе на системы навигации.


**Входные данные**

- `solar_wind`: датафрейм с данными солнечного ветра.
-	`time_tag`: время измерения, представленное в формате временной метки.
-	`speed`: скорость солнечного ветра (км/с).
-	`density`: плотность солнечного ветра (частицы/см³).
-	`temperature`: температура солнечного ветра (K).

**Что нужно сделать**

Построй график с тремя параметрами солнечного ветра на одной фигуре с раздельными осями:

-	В первой строке графика отобрази скорость солнечного ветра с красной линией.
-	Во второй строке отобрази плотность солнечного ветра с зелёной линией.
-	В третьей строке отобрази температуру солнечного ветра с синей линией.

**Требования**

-	Ось $X$ должна быть общей для всех трёх графиков.
-	Оси $Y$ должны быть разные для каждого графика.
-	Линии на графиках должны содержать маркеры для каждой точки.
-	Заголовки для каждого подграфика:
  - «Speed» для первой строки.
  -	«Density» для второй строки.
  -	«Temperature» для третьей строки.
  -	Общий заголовок для фигуры: «Solar Wind Parameters over Time».
-	Размер фигуры — 800x1000 пикселей.

**Примечание.** Может показаться, что в Plotly Express сложно поменять цвет линии графика. В таком случае загляни в лонгрид. Все необходимые инструменты описаны в разделе «Анатомия графика».

In [ ]:
# Напииши свой код здесь


## Задача 4

Построй анимированную карту землетрясений. Для этого используй библиотеку `Plotly Express`, функцию `scatter_geo` и анимацию, которая отображает землетрясения на протяжении определённого времени.

**Входные данные**
- `earthquake`: датафрейм с данными.
-	`longitude`: долгота эпицентра землетрясения.
-	`latitude`: широта эпицентра землетрясения.
-	`mag`: магнитуда землетрясения (чем выше значение, тем сильнее землетрясение).
-	`time`: временная метка землетрясения.

**Что нужно сделать**

- Создай график, где анимация будет отображать землетрясения по временной шкале.
- Используй функцию `scatter_geo`, чтобы отобразить точки землетрясений по их координатам на карте с проекцией `natural earth`.
- Зафиксируй цветовую шкалу в диапазоне значений магнитуды от 0 до 10, используя параметр `range_color=[0, 10]`.
- Настрой цвет и размер маркеров:
   - Максимальный размер маркеров — 20.
   - Размер маркеров и их цвет должны соответствовать величине землетрясения. Более сильные землетрясения должны быть большего размера и темнее по цвету.


In [ ]:
# Напииши свой код здесь


## Задача 5

Прогнозирование северного сияния помогает изучать солнечную активность и взаимодействие солнечного ветра с магнитным полем Земли. Это позволяет лучше понять физические процессы, происходящие в атмосфере и в космосе.

Построй визуализацию прогноза появления северного сияния с помощью `px.scatter_geo`. График должен отображать интенсивность сияния в зависимости от географических координат.

**Входные данные**
- `aurora`: датафрейм с данными.
- `Observation Time`: время наблюдения (например, `2024-09-24T10:21:00Z`).
- `Forecast Time`: время прогнозируемых данных (например, `2024-09-24T11:19:00Z`).
- `Data Format`: структура данных, представляющая собой массив [Longitude, Latitude, Aurora], где каждый элемент массива состоит из трёх значений:
  -	`Longitude`: долгота (географическая координата).
  -	`Latitude`: широта (географическая координата).
  -	`Aurora`: числовое значение, указывающее на интенсивность сияния в данном месте.

**Что нужно сделать**

-	Отфильтруй данные так, чтобы на график попали только те точки, где индекс сияния Aurora больше 10.
-	Построй график с проекцией `orthographic`.
- Установи цветовую палитру от светло-жёлтых значений при слабой интенсивности до тёмно-красных при большой вероятности появления сияния.
- Добавь подписи и заголовок к графику. Название графика: «Aurora Data Visualization».


In [ ]:
# Напииши свой код здесь
